<h1 align="center"> Aplicações em Processamento de Linguagem Natural </h1>
<h2 align="center"> Aula 04 - Classificação de Texto</h2>
<h3 align="center"> Prof. Fernando Vieira da Silva MSc.</h3>

<h2>Problema de Classificação</h2>

<p>Neste tutorial vamos trabalhar com um exemplo prático de problema de classificação de texto. O objetivo é identificar uma sentença como escrita "formal" ou "informal".</p>

<b>1. Obtendo o corpus</b>

<p>Para simplificar o problema, vamos continuar utilizando o corpus Gutenberg como textos formais e vamos usar mensagens de chat do corpus <b>nps_chat</b> como textos informais.</p>
<p>Antes de tudo, vamos baixar o corpus nps_chat:</p>

In [ ]:
import nltk

nltk.download('nps_chat')

In [ ]:
from nltk.corpus import nps_chat

print(nps_chat.fileids())

<p>Agora vamos ler os dois corpus e armazenar as sentenças em uma mesma ndarray. Perceba que também teremos uma ndarray para indicar se o texto é formal ou não. Começamos armazenando o corpus em lists. Vamos usar apenas 500 elementos de cada, para fins didáticos.</p>

In [ ]:
import nltk

x_data_nps = []

for fileid in nltk.corpus.nps_chat.fileids():#corre os filtros
    x_data_nps.extend([post.text for post in nps_chat.xml_posts(fileid)])

y_data_nps = [0] * len(x_data_nps)#CRIAR MATRIZ DE 0 do mesmo tamanho q o dataset

x_data_gut = []
#uni as palavras pra montar a sentença depois tokeniza
for fileid in nltk.corpus.gutenberg.fileids():
    x_data_gut.extend([' '.join(sent) for sent in nltk.corpus.gutenberg.sents(fileid)])#.sents-segmentsdo
    
y_data_gut = [1] * len(x_data_gut)

x_data_full = x_data_nps[:500] + x_data_gut[:500]
print(len(x_data_full))
y_data_full = y_data_nps[:500] + y_data_gut[:500]
print(len(y_data_full))

In [ ]:
#10 primeiros
print(x_data_full[:10])
print(y_data_full[:10])

In [ ]:
#10 últimos
print(x_data_full[-10:])
print(y_data_full[-10:])

<p>Em seguida, transformamos essas listas em ndarrays, para usarmos nas etapas de pré-processamento que já conhecemos.</p>

In [ ]:
import numpy as np

x_data = np.array(x_data_full, dtype=object)
#x_data = np.array(x_data_full)
print(x_data.shape)
y_data = np.array(y_data_full)
print(y_data.shape)

<b>2. Dividindo em datasets de treino e teste</b>

<p>Para que a pesquisa seja confiável, precisamos avaliar os resultados em um dataset de teste. Por isso, vamos dividir os dados aleatoriamente, deixando 80% para treino e o demais para testar os resultados em breve.</p>

In [ ]:
train_indexes = np.random.rand(len(x_data)) < 0.80

print(len(train_indexes))
print(train_indexes[:10])

In [ ]:
x_data_train = x_data[train_indexes]
y_data_train = y_data[train_indexes]

print(len(x_data_train))
print(len(y_data_train))

In [ ]:
x_data_test = x_data[~train_indexes]
y_data_test = y_data[~train_indexes]

print(len(x_data_test))
print(len(y_data_test))

<b>3. Treinando o classificador</b>

<p>Para tokenização, vamos usar a mesma função do tutorial anterior:</p>

In [ ]:
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import wordnet

stopwords_list = stopwords.words('english')

lemmatizer = WordNetLemmatizer()

def my_tokenizer(doc):#doc=uma sentença
    words = word_tokenize(doc)
    
    pos_tags = pos_tag(words)
    
    non_stopwords = [w for w in pos_tags if not w[0].lower() in stopwords_list]
    
    non_punctuation = [w for w in non_stopwords if not w[0] in string.punctuation]
    
    lemmas = []
    for w in non_punctuation:#converte os postab para o que tem no wordnet
        if w[1].startswith('J'):
            pos = wordnet.ADJ
        elif w[1].startswith('V'):
            pos = wordnet.VERB
        elif w[1].startswith('N'):
            pos = wordnet.NOUN
        elif w[1].startswith('R'):
            pos = wordnet.ADV
        else:
            pos = wordnet.NOUN
        
        lemmas.append(lemmatizer.lemmatize(w[0], pos))

    return lemmas
    
    

<p>Mas agora vamos criar um <b>pipeline</b> contendo o vetorizador TF-IDF, o SVD para redução de atributos e um algoritmo de classificação. Mas antes, vamos encapsular nosso algoritmo para escolher o número de dimensões para o SVD em uma classe que pode ser utilizada com o pipeline:</p>

In [ ]:
from sklearn.decomposition import TruncatedSVD

class SVDDimSelect(object):
    def fit(self, X, y=None):        
        try:
            self.svd_transformer = TruncatedSVD(n_components=round(X.shape[1]/2))#2000 componentes
            self.svd_transformer.fit(X)
        
            cummulative_variance = 0.0
            k = 0
            for var in sorted(self.svd_transformer.explained_variance_ratio_)[::-1]:#ordenado inverso(do maior para o menor)r)
                cummulative_variance += var
                if cummulative_variance >= 0.5:#50% da representação da variância no SVD
                    break
                else:
                    k += 1
                
            self.svd_transformer = TruncatedSVD(n_components=k)
        except Exception as ex:
            print(ex)
            
        return self.svd_transformer.fit(X)
    
    def transform(self, X, Y=None):
        return self.svd_transformer.transform(X)
        
    def get_params(self, deep=True):
        return {}

<p>Finalmente podemos criar nosso pipeline:</p>

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn import neighbors

clf = neighbors.KNeighborsClassifier(n_neighbors=10, weights='uniform')

my_pipeline = Pipeline([('tfidf', TfidfVectorizer(tokenizer=my_tokenizer)),\
                       ('svd', SVDDimSelect()), \
                       ('clf', clf)])

<p>Estamos quase lá... Agora vamos criar um objeto <b>RandomizedSearchCV</b> que fará a seleção de hiper-parâmetros do nosso classificador (aka. parâmetros que não são aprendidos durante o treinamento). Essa etapa é importante para obtermos a melhor configuração do algoritmo de classificação. Para economizar tempo de treinamento, vamos usar um algoritmo simples o <i>K nearest neighbors (KNN)</i>.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import scipy

par = {'clf__n_neighbors': range(1, 60), 'clf__weights': ['uniform', 'distance']}


hyperpar_selector = RandomizedSearchCV(my_pipeline, par, cv=3, scoring='accuracy', n_jobs=2, n_iter=20)#cv = 3 folders do cross validation
#n_jobs = n° de threads, n_iter = 20 parâmetros

<p>E agora vamos treinar nosso algoritmo, usando o pipeline com seleção de atributos:</p>

In [ ]:
print(x_data_train)

In [ ]:
hyperpar_selector.fit(X=x_data_train, y=y_data_train)

In [ ]:
print("Best score: %0.3f" % hyperpar_selector.best_score_)
print("Best parameters set:")
best_parameters = hyperpar_selector.best_estimator_.get_params()
for param_name in sorted(par.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

<b>4. Testando o classificador</b>

<p>Agora vamos usar o classificador com o nosso dataset de testes, e observar os resultados:</p>

In [ ]:
from sklearn.metrics import *

y_pred = hyperpar_selector.predict(x_data_test)

print(accuracy_score(y_data_test, y_pred))

<b>5. Serializando o modelo</b><br>

In [ ]:
import pickle

string_obj = pickle.dumps(hyperpar_selector)

In [ ]:
model_file = open('model.pkl', 'wb')

model_file.write(string_obj)

model_file.close()

<b>6. Abrindo e usando um modelo salvo </b><br>

In [ ]:

model_file = open('model.pkl', 'rb')
model_content = model_file.read()

obj_classifier = pickle.loads(model_content)

model_file.close()

res = obj_classifier.predict(["what's up bro?"])

print(res)

In [ ]:
res = obj_classifier.predict(x_data_test)
print(accuracy_score(y_data_test, res))

In [ ]:
res = obj_classifier.predict(x_data_test)

print(res)

In [ ]:
formal = [x_data_test[i] for i in range(len(res)) if res[i] == 1]

for txt in formal:
    print("%s\n" % txt)


In [ ]:
informal = [x_data_test[i] for i in range(len(res)) if res[i] == 0]

for txt in informal:
    print("%s\n" % txt)

In [ ]:
res2 = obj_classifier.predict(["Emma spared no exertions to maintain this happier flow of ideas , and hoped , by the help of backgammon , to get her father tolerably through the evening , and be attacked by no regrets but her own"])

print(res2)

<p><b>Exercício 4:</b>  Treine um modelo para classificar sentenças de acordo com o sentimento (positivo ou negativo), utilizando o corpus sentence_polarity do nltk.
    Dica: Para obter sentenças positivas use:
</p>

In [15]:
import warnings
warnings.filterwarnings("ignore")

import nltk
nltk.download('sentence_polarity')
from nltk.corpus import sentence_polarity

sentence_polarity.sents(categories=['pos'])

[nltk_data] Downloading package sentence_polarity to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package sentence_polarity is already up-to-date!


[['the', 'rock', 'is', 'destined', 'to', 'be', 'the', '21st', "century's", 'new', '"', 'conan', '"', 'and', 'that', "he's", 'going', 'to', 'make', 'a', 'splash', 'even', 'greater', 'than', 'arnold', 'schwarzenegger', ',', 'jean-claud', 'van', 'damme', 'or', 'steven', 'segal', '.'], ['the', 'gorgeously', 'elaborate', 'continuation', 'of', '"', 'the', 'lord', 'of', 'the', 'rings', '"', 'trilogy', 'is', 'so', 'huge', 'that', 'a', 'column', 'of', 'words', 'cannot', 'adequately', 'describe', 'co-writer/director', 'peter', "jackson's", 'expanded', 'vision', 'of', 'j', '.', 'r', '.', 'r', '.', "tolkien's", 'middle-earth', '.'], ...]

In [16]:
print(len(sentence_polarity.sents(categories=['neg'])))
print(len(sentence_polarity.sents(categories=['pos'])))

5331
5331


In [17]:
import nltk

#x_data_neg = sentence_polarity.sents(categories=['neg'])
x_data_neg = []
x_data_neg.extend([' '.join(sent) for sent in sentence_polarity.sents(categories=['neg'])])

y_data_neg = [0] * len(sentence_polarity.sents(categories=['neg']))

#x_data_pos = sentence_polarity.sents(categories=['pos'])
x_data_pos = []
x_data_pos.extend([' '.join(sent) for sent in sentence_polarity.sents(categories=['pos'])])

y_data_pos = [1] * len(sentence_polarity.sents(categories=['pos']))

x_data_full = x_data_neg[:1000] + x_data_pos[:1000]
print(len(x_data_full))
y_data_full = y_data_neg[:1000] + y_data_pos[:1000]
print(len(y_data_full))

2000
2000


In [18]:
import numpy as np

x_data = np.array(x_data_full, dtype=object)
#x_data = np.array(x_data_full)
print(x_data.shape)
y_data = np.array(y_data_full)
print(y_data.shape)

(2000,)
(2000,)


In [19]:
train_indexes = np.random.rand(len(x_data)) < 0.80

print(len(train_indexes))
print(train_indexes[:10])

2000
[ True False  True  True  True  True False False  True  True]


In [20]:
x_data_train = x_data[train_indexes]
y_data_train = y_data[train_indexes]

print(len(x_data_train))
print(len(y_data_train))

1604
1604


In [21]:
x_data_test = x_data[~train_indexes]
y_data_test = y_data[~train_indexes]

print(len(x_data_test))
print(len(y_data_test))

396
396


In [22]:
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import wordnet

stopwords_list = stopwords.words('english')

lemmatizer = WordNetLemmatizer()

def my_tokenizer(doc):#doc=uma sentença
    words = word_tokenize(doc)
    
    pos_tags = pos_tag(words)
    
    non_stopwords = [w for w in pos_tags if not w[0].lower in stopwords_list]
    
    non_punctuation = [w for w in non_stopwords if not w[0] in string.punctuation]
    
    lemmas = []
    for w in non_punctuation:#converte os postab para o que tem no wordnet
        if w[1].startswith('J'):
            pos = wordnet.ADJ
        elif w[1].startswith('V'):
            pos = wordnet.VERB
        elif w[1].startswith('N'):
            pos = wordnet.NOUN
        elif w[1].startswith('R'):
            pos = wordnet.ADV
        else:
            pos = wordnet.NOUN
        
        lemmas.append(lemmatizer.lemmatize(w[0], pos))

    return lemmas
    
    

In [23]:
from sklearn.decomposition import TruncatedSVD

class SVDDimSelect(object):
    def fit(self, X, y=None):        
        try:
            self.svd_transformer = TruncatedSVD(n_components=round(X.shape[1]/2))#2000 componentes
            self.svd_transformer.fit(X)
        
            cummulative_variance = 0.0
            k = 0
            for var in sorted(self.svd_transformer.explained_variance_ratio_)[::-1]:#ordenado inverso(do maior para o menor)r)
                cummulative_variance += var
                if cummulative_variance >= 0.8:#50% da representação da variância no SVD
                    break
                else:
                    k += 1
                
            self.svd_transformer = TruncatedSVD(n_components=k)
        except Exception as ex:
            print(ex)
            
        return self.svd_transformer.fit(X)
    
    def transform(self, X, Y=None):
        return self.svd_transformer.transform(X)
        
    def get_params(self, deep=True):
        return {}

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn import neighbors

clf = neighbors.KNeighborsClassifier(n_neighbors=10, weights='uniform')

my_pipeline = Pipeline([('tfidf', TfidfVectorizer(tokenizer=my_tokenizer)),\
                       ('svd', SVDDimSelect()), \
                       ('clf', clf)])

In [25]:
from sklearn.model_selection import RandomizedSearchCV
import scipy

par = {'clf__n_neighbors': range(1, 100), 'clf__weights': ['uniform', 'distance']}


hyperpar_selector = RandomizedSearchCV(my_pipeline, par, cv=3, scoring='accuracy', n_jobs=1, n_iter=40)#cv = 3 folders do cross validation


In [26]:
hyperpar_selector.fit(X=x_data_train, y=y_data_train)

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...i',
           metric_params=None, n_jobs=None, n_neighbors=10, p=2,
           weights='uniform'))]),
          fit_params=None, iid='warn', n_iter=40, n_jobs=1,
          param_distributions={'clf__n_neighbors': range(1, 100), 'clf__weights': ['uniform', 'distance']},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='accuracy', verbose=0)

In [27]:

print("Best score: %0.3f" % hyperpar_selector.best_score_)
print("Best parameters set:")
best_parameters = hyperpar_selector.best_estimator_.get_params()
for param_name in sorted(par.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best score: 0.658
Best parameters set:
	clf__n_neighbors: 58
	clf__weights: 'distance'


In [28]:
from sklearn.metrics import *

y_pred = hyperpar_selector.predict(x_data_test)

print(accuracy_score(y_data_test, y_pred))

0.648989898989899


In [ ]:
#aumentar os dados
#aumentar a variância
#aumentar os parâmetros
#aumentar o range dos parâmetros